In [124]:
from cobra.io import load_json_model, read_sbml_model, load_matlab_model
import cobra
import numpy as np
import pandas as pd
from scipy.cluster import hierarchy
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
import itertools
from cobra.flux_analysis import flux_variability_analysis

In [116]:
ec_core_cobra_model = read_sbml_model("/home/touliopoulos/project/erasmus/erasmus_2025_project/data/Reconstructions_data/e_coli_core.xml")
cobra_model_reactions = [r.id for r in ec_core_cobra_model.reactions]

biomass_index = cobra_model_reactions.index("BIOMASS_Ecoli_core_w_GAM")
print(ec_core_cobra_model.reactions.get_by_id("BIOMASS_Ecoli_core_w_GAM").bounds)


(0.0, 1000.0)


In [117]:
fba_solution = ec_core_cobra_model.optimize()
objective_value = fba_solution.objective_value
objective_value = int(objective_value * 100) / 100 # 0.87

ec_core_cobra_model_100 = ec_core_cobra_model.copy()
ec_core_cobra_model_50 = ec_core_cobra_model.copy()
ec_core_cobra_model_0 = ec_core_cobra_model.copy()

#ec_core_cobra_model_100.reactions.get_by_id("BIOMASS_Ecoli_core_w_GAM").bounds = (objective_value, 1000.0)
#ec_core_cobra_model_50.reactions.get_by_id("BIOMASS_Ecoli_core_w_GAM").bounds = (objective_value/(100/50), 1000.0)
#ec_core_cobra_model_0.reactions.get_by_id("BIOMASS_Ecoli_core_w_GAM").bounds = (objective_value/(100/1e-9), 1000.0)


#ec_core_cobra_model_100.reactions.get_by_id("BIOMASS_Ecoli_core_w_GAM").bounds = (0, objective_value)
#ec_core_cobra_model_50.reactions.get_by_id("BIOMASS_Ecoli_core_w_GAM").bounds = (0, (objective_value/(100/50)))
#ec_core_cobra_model_0.reactions.get_by_id("BIOMASS_Ecoli_core_w_GAM").bounds = (0, (objective_value/(100/1e-9)))


In [76]:
sampler_100 = cobra.sampling.optgp.OptGPSampler(ec_core_cobra_model_100)
samples_100 = sampler_100.sample(n=1000)

sampler_50 = cobra.sampling.optgp.OptGPSampler(ec_core_cobra_model_50)
samples_50 = sampler_50.sample(n=1000)

sampler_0 = cobra.sampling.optgp.OptGPSampler(ec_core_cobra_model_0)
samples_0 = sampler_0.sample(n=1000)

In [77]:
samples_100 = abs(samples_100.to_numpy()).T
samples_50 = abs(samples_50.to_numpy()).T
samples_0 = abs(samples_0.to_numpy()).T

#print(samples_100.shape)
#print(type(samples_100))

print(np.max(samples_100[biomass_index]))
print(np.min(samples_100[biomass_index]))

print(np.max(samples_50[biomass_index]))
print(np.min(samples_50[biomass_index]))

print(np.max(samples_0[biomass_index]))
print(np.min(samples_0[biomass_index]))

0.8709883644313385
0.8700004057975557
0.5831490614181757
0.4350074398756605
0.23858060341330034
0.000137642556488586


In [78]:

correlation_matrix_100 = np.corrcoef(samples_100)
correlation_matrix_100[np.isnan(correlation_matrix_100)] = 0
print(correlation_matrix_100.shape)


correlation_matrix_50 = np.corrcoef(samples_50)
correlation_matrix_50[np.isnan(correlation_matrix_50)] = 0
print(correlation_matrix_50.shape)


correlation_matrix_0 = np.corrcoef(samples_0)
correlation_matrix_0[np.isnan(correlation_matrix_0)] = 0
print(correlation_matrix_0.shape)


(95, 95)
(95, 95)
(95, 95)


/home/touliopoulos/miniconda3/envs/py38/lib/python3.8/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning:

invalid value encountered in divide

/home/touliopoulos/miniconda3/envs/py38/lib/python3.8/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning:

invalid value encountered in divide



In [89]:

def cluster_corr_reactions(correlation_matrix, reactions, linkage="ward", 
                           t = 2.0, correction=True):
    
    # function to return a nested list with grouped reactions based on clustering
    def clusters_list(reactions, labels):
        clusters = []
        unique_labels = np.unique(labels)
        for label in unique_labels:
            cluster = []
            label_where = np.where(labels == label)[0]
            for where in label_where:
                cluster.append(reactions[where])
            clusters.append(cluster)
        return clusters
    
    if correction == True:        
        dissimilarity_matrix = 1 - abs(correlation_matrix)
    else:
        dissimilarity_matrix = 1 - correlation_matrix
            
    Z = hierarchy.linkage(dissimilarity_matrix, linkage)
    labels = hierarchy.fcluster(Z, t, criterion='distance')
    
    clusters = clusters_list(reactions, labels)
    return dissimilarity_matrix, labels, clusters



dissimilarity_matrix, labels, clusters = cluster_corr_reactions(correlation_matrix_100, cobra_model_reactions)
print(clusters)

dissimilarity_matrix, labels, clusters = cluster_corr_reactions(correlation_matrix_50, cobra_model_reactions)
print(clusters)

dissimilarity_matrix, labels, clusters = cluster_corr_reactions(correlation_matrix_0, cobra_model_reactions)
print(clusters)




[['PYK', 'MDH', 'PDH'], ['PPC', 'AKGDH', 'SUCOAS', 'ICDHyr', 'ICL', 'MALS'], ['PFK', 'PGI', 'PGK', 'PGL', 'PGM', 'ACONTa', 'ACONTb', 'RPE', 'CS', 'RPI', 'ENO', 'TALA', 'TKT1', 'TKT2', 'TPI', 'FBA', 'FUM', 'G6PDH2r', 'GAPD', 'GND'], ['ATPS4r', 'NADH16'], ['THD2', 'NADTRHD'], ['PFL', 'EX_for_e', 'EX_h_e'], ['CO2t', 'CYTBD', 'EX_co2_e', 'EX_h2o_e', 'EX_o2_e', 'H2Ot', 'O2t'], ['ACKr', 'ACt2r', 'PTAr', 'EX_ac_e'], ['EX_glu__L_e', 'EX_nh4_e', 'GLUt2r', 'NH4t'], ['ALCD2x', 'ETOHt2r', 'EX_etoh_e'], ['ACALD', 'ACALDt', 'EX_acald_e'], ['D_LACt2', 'EX_lac__D_e', 'LDH_D'], ['PIt2r', 'BIOMASS_Ecoli_core_w_GAM', 'EX_pi_e'], ['GLNS', 'GLUDy', 'GLUSy'], ['PPS', 'ADK1'], ['FORt2', 'FORt'], ['SUCDi', 'FRD7'], ['EX_fru_e', 'EX_fum_e', 'EX_gln__L_e', 'EX_mal__L_e', 'EX_succ_e', 'FRUpts2', 'FUMt2_2', 'GLNabc', 'MALt2_2'], ['ATPM', 'PPCK', 'FBP', 'GLUN', 'ME1', 'ME2'], ['PYRt2', 'EX_pyr_e'], ['SUCCt2_2', 'SUCCt3'], ['AKGt2r', 'EX_akg_e'], ['EX_glc__D_e', 'GLCpts']]
[['PGK', 'PGL', 'PGM', 'RPE', 'RPI', 'ENO'

In [120]:


def add_pFBA_constraint(model, objective, objective_fraction=1, flux_fraction=1):
    """Add a pFBA flux minimality constraint to the model. Updates the model in-place.

    :param model: The cobra model
    :type model: cobra.Model
    :param objective: The objective that should be maximized,
    in order to determine the objective bound for the flux sum minimization.
    :type objective: cobra.Reaction
    :param objective_fraction: pFBA minimum objective fraction, should be 1<= and defaults to 1.
    :type objective_fraction: float, optional
    :param flux_fraction: pFBA maximum flux fraction, should be >=1 and defaults to 1.
    :type flux_fraction: float, optional
    
    :return: The model with the flux minimality constraint added.
    :rtype: cobra.Model
    """

    def recursive_sum(x):
        """Recursive version of sum.

        Sum operations are slow on variables. By doing it recursively, we can use
        2log(length(x)) additions instead of length(x).
        We could exceed the recursion depth, but it is not very likely with normal model sizes.
        """
        length = len(x)
        if length == 1:
            return x[0]
        elif length == 2:
            return x[0] + x[1]
        else:
            split = length // 2
            return recursive_sum(x[split:]) + recursive_sum(x[:split])

    # Calculate flux sum for the objective.
    flux_sum_value = cobra.flux_analysis.pfba(model, objective_fraction, objective)

    # Create a total flux variable (fw + rev fluxes, same as pFBA does it in Cobra.)
    variables = ((r.forward_variable, r.reverse_variable) for r in model.reactions)
    sum_variable = recursive_sum(list(itertools.chain.from_iterable(variables)))
    # Create and add the constraint to the model.
    constraint = model.problem.Constraint(
        expression=sum_variable,
        name="total_flux_sum_bound",
        lb=0,
        ub=flux_sum_value.objective_value * flux_fraction,
    )
    model.add_cons_vars([constraint])
    return model



ec_core_cobra_model_100_pfba = ec_core_cobra_model_100.copy()
ec_core_cobra_model_100_pfba = add_pFBA_constraint(ec_core_cobra_model_100_pfba, ec_core_cobra_model_100_pfba.reactions.get_by_id("BIOMASS_Ecoli_core_w_GAM"), 1, 1.25)


ec_core_cobra_model_0_pfba = ec_core_cobra_model_0.copy()
ec_core_cobra_model_0_pfba = add_pFBA_constraint(ec_core_cobra_model_0_pfba, ec_core_cobra_model_0_pfba.reactions.get_by_id("BIOMASS_Ecoli_core_w_GAM"), 0, 1.25)


In [ ]:

solution = ec_core_cobra_model_100_pfba.optimize()
print(solution.get_primal_by_id("FRD7"))
print(solution.get_primal_by_id("SUCDi"))
print(flux_variability_analysis(ec_core_cobra_model_100_pfba, ec_core_cobra_model_100_pfba.reactions))


solution = ec_core_cobra_model_100.optimize()
print(solution.get_primal_by_id("FRD7"))
print(solution.get_primal_by_id("SUCDi"))

solution = ec_core_cobra_model_0_pfba.optimize()
print(solution.get_primal_by_id("FRD7"))
print(solution.get_primal_by_id("SUCDi"))

solution = ec_core_cobra_model_0.optimize()
print(solution.get_primal_by_id("FRD7"))
print(solution.get_primal_by_id("SUCDi"))

#sampler_100 = cobra.sampling.optgp.OptGPSampler(ec_core_cobra_model_100)
#samples_100 = sampler_100.sample(n=1000)


0.0
5.064375661482114
0.0
5.064375661482093
0.0
1.1205812344987952e-15
0.0
5.064375661482093
